# Obstacle Avoidance

## Aseba code

var speed0 = 100       # nominal speed  
var speedGain = 2      # gain used with ground gradient  
var obstThrL = 10      # low obstacle threshold to switch state 1->0  
var obstThrH = 20      # high obstacle threshold to switch state 0->1  
var obstSpeedGain = 5  # /100 (actual gain: 5/100=0.05)  
  
var state = 0          # 0=gradient, 1=obstacle avoidance  
var diffDelta          # difference between ground sensors  
var obst[2]            # measurements from left and right prox sensors  
  
timer.period[0] = 10   # 10ms sampling time  
  
onevent timer0  
  .# acquisition from ground sensor for going toward the goal  
  diffDelta = prox.ground.delta[1] - prox.ground.delta[0]  
  .# acquisition from the proximity sensors to detect obstacles  
  obst = [prox.horizontal[0], prox.horizontal[4]]  
  if state == 0 and (obst[0] > obstThrH or obst[1] > obstThrH) then  
    # switch from goal tracking to obst avoidance if obstacle detected  
    state = 1  
  elseif state == 1 and obst[0] < obstThrL and obst[1] < obstThrL then  
    # switch from obst avoidance to goal tracking if obstacle got unseen  
    state = 0  
  end  
  if  state == 0 then  
    # goal tracking: turn toward the goal  
    motor.left.target = speed0 - speedGain * diffDelta  
    motor.right.target = speed0 + speedGain * diffDelta  
  else  
    # obstacle avoidance: accelerate wheel near obstacle  
    motor.left.target = speed0 + obstSpeedGain * (obst[0] / 100)  
    motor.right.target = speed0 + obstSpeedGain * (obst[1] / 100)  
  end  

## Python code

### Imports

In [80]:
import os
import sys
import time
import serial
#import math
import numpy as np
#from scipy.interpolate import interp1d

# Adding the src folder in the current directory as it contains the script for Thymio and local occupancy
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio
#from local_occupancy import sensor_measurements, sensor_distances
#from local_occupancy import thymio_coords, sensor_pos_from_center, sensor_angles

In [81]:
robot = Thymio.serial(port='COM3', refreshing_rate=0.1)

### Accessible functions for the robot

In [8]:
#dir(robot)

### Read/Write variables

In [9]:
#variables = robot.variable_description()
#for var in variables :
#    print(var)

### Variables globales

In [82]:
PROXIMITY_THRESHOLD = 3800 #around 2cm
TIME_SLEEP_OBSTACLE = 0.1 #in seconds
TIME_SLEEP_DATA = 0.2 #5Hz frequency
MOTOR_SPEED = 80 #both motor are set with this value (one in each direction)
#WEIGHTS = [0,1,2,3,4,0,0]

SENSOR_0 = 0
SENSOR_1 = 1
SENSOR_2 = 2
SENSOR_3 = 3
SENSOR_4 = 4
SENSOR_5 = 5
SENSOR_6 = 6
ALL_FRONT_SENSOR = 5
PARTIALY_ALL_FRONT_SENSOR = 4

### Main function 

In [87]:
for i in range(1000000): #simulation of big while loop
    robot.set_var("motor.left.target", MOTOR_SPEED)        #TO REMOVE
    robot.set_var("motor.right.target", MOTOR_SPEED)       #TO REMOVE
    
    sensor_measurment = robot["prox.horizontal"]
    
    if np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)):
        pass
    else:
        obstacle_direction = np.nonzero(sensor_measurment)[0]
        while((np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))and(np.all(np.not_equal(obstacle_direction, SENSOR_5)))and(np.all(np.not_equal(obstacle_direction, SENSOR_6)))):
            if(np.shape(obstacle_direction)[0] == ALL_FRONT_SENSOR)or(np.shape(obstacle_direction)[0] == PARTIALY_ALL_FRONT_SENSOR):
                if(sensor_measurment[SENSOR_1]>sensor_measurment[SENSOR_3]):
                    robot.set_var("motor.left.target", MOTOR_SPEED)
                    robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                else:
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
            
            else:
                if (np.any(np.equal(obstacle_direction,SENSOR_0)))or(np.any(np.equal(obstacle_direction,SENSOR_1))):
                    robot.set_var("motor.left.target", MOTOR_SPEED)
                    robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                elif (np.any(np.equal(obstacle_direction,SENSOR_3)))or(np.any(np.equal(obstacle_direction,SENSOR_4))):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
                elif (np.any(np.equal(obstacle_direction,SENSOR_2))):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
        
            time.sleep(TIME_SLEEP_OBSTACLE)
            
            sensor_measurment = robot["prox.horizontal"]
            obstacle_direction = np.nonzero(sensor_measurment)[0]
        
    time.sleep(TIME_SLEEP_OBSTACLE)
robot.set_var("motor.left.target", MOTOR_SPEED)   #TO REMOVE
robot.set_var("motor.right.target", MOTOR_SPEED)  #TO REMOVE

KeyboardInterrupt: 

### Tests

ANN

In [85]:
WEIGHT_L=[40,20,-20,-20,-40, 0,0] #sensor_5 = 30 ; sensor = -10
WEIGHT_R=[-40,-20,-20,20,40, 0,0] #sensor_5 = -10 ; sensor = 30
SENSOR_SCALE = 4000
THRESHOLD = 20

for i in range(10000000): #simulation of big while loop 
    
    robot.set_var("motor.left.target", MOTOR_SPEED)        #TO REMOVE
    robot.set_var("motor.right.target", MOTOR_SPEED)       #TO REMOVE
    
    sensor_measurment = robot["prox.horizontal"]
    print(sensor_measurment)
    
    measurement_rescaled = np.divide(sensor_measurment, SENSOR_SCALE)
    #print(measurement_rescaled)
    weighted_measurment_l = np.multiply(measurement_rescaled, WEIGHT_L)
    #print(weighted_measurment_l)
    weighted_measurment_r = np.multiply(measurement_rescaled, WEIGHT_R)
    #print(weighted_measurment_r)

    motor_speed_l = np.sum(weighted_measurment_l)
    motor_speed_r = np.sum(weighted_measurment_r)

    print(motor_speed_l)
    print(motor_speed_r)
    
    if np.any(np.greater(motor_speed_l, THRESHOLD)) or np.any(np.greater(motor_speed_r, THRESHOLD)):       
        
        if motor_speed_l<0:
            robot.set_var("motor.left.target", 2**16 + int(motor_speed_l))
        else :
            robot.set_var("motor.left.target", int(motor_speed_l))
     
        if motor_speed_r<0:
            robot.set_var("motor.right.target", 2**16 + int(motor_speed_r))
        else :
            robot.set_var("motor.right.target", int(motor_speed_r))
        
        
        """
        robot.set_var("motor.left.target", 0)
        robot.set_var("motor.right.target", 0)
        
        robot.set_var("motor.left.target", 2**16-motor_speed_l)
        robot.set_var("motor.right.target", 2**16-motor_speed_r)
        """
    
    time.sleep(TIME_SLEEP_DATA)

    

[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 

[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 

0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0,

0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0, 0, 0, 0, 0, 0, 0]
0.0
0.0
[0

[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999995
-7.1899999999999995
[0, 1438, 0, 0, 0, 0, 0]
7.1899999999999

...

[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2438]


[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]
0.0
0.0
[0, 0, 0, 0, 0, 2534, 2457]


0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 2455]
0.0
0.0
[0, 0, 0, 0, 0, 2549, 24

[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]
0.0
0.0
[0, 0, 0, 0, 0, 2548, 2462]


KeyboardInterrupt: 

If/elif

In [89]:
for i in range(1000000): #simulation of big while loop
    robot.set_var("motor.left.target", MOTOR_SPEED)        #TO REMOVE
    robot.set_var("motor.right.target", MOTOR_SPEED)       #TO REMOVE
    
    sensor_measurment = robot["prox.horizontal"]
    print(sensor_measurment)                               #TO REMOVE
    
    if np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)):
        pass
        #break ?
    else:
        obstacle_direction = np.nonzero(sensor_measurment)[0]
        
        while((np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))and(np.all(np.not_equal(obstacle_direction, SENSOR_5)))and(np.all(np.not_equal(obstacle_direction, SENSOR_6)))):
            if(np.shape(obstacle_direction)[0] == ALL_FRONT_SENSOR)or(np.shape(obstacle_direction)[0] == PARTIALY_ALL_FRONT_SENSOR):
                if(sensor_measurment[SENSOR_1]>sensor_measurment[SENSOR_3]):
                    robot.set_var("motor.left.target", MOTOR_SPEED)
                    robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                else:
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
            
            else:
                if (np.any(np.equal(obstacle_direction,SENSOR_0)))or(np.any(np.equal(obstacle_direction,SENSOR_1))):
                    robot.set_var("motor.left.target", MOTOR_SPEED)
                    robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                elif (np.any(np.equal(obstacle_direction,SENSOR_3)))or(np.any(np.equal(obstacle_direction,SENSOR_4))):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
                elif (np.any(np.equal(obstacle_direction,SENSOR_2))):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
        
            print(sensor_measurment)                      #TO REMOVE
            time.sleep(TIME_SLEEP_OBSTACLE)
            
            sensor_measurment = robot["prox.horizontal"]
            obstacle_direction = np.nonzero(sensor_measurment)[0]
        
    time.sleep(TIME_SLEEP_DATA)
robot.set_var("motor.left.target", MOTOR_SPEED)   #TO REMOVE
robot.set_var("motor.right.target", MOTOR_SPEED)  #TO REMOVE

[0, 0, 2710, 0, 0, 0, 2317]
[0, 0, 2726, 0, 0, 0, 2343]
[0, 0, 3164, 0, 0, 0, 2421]
[0, 0, 3513, 0, 0, 0, 2414]
[0, 0, 3736, 0, 0, 0, 2397]
[0, 0, 4112, 1182, 0, 0, 2372]
[0, 0, 4223, 2055, 0, 0, 2311]
[0, 1357, 4255, 2655, 0, 0, 2233]
[0, 1891, 4170, 3530, 0, 0, 2110]
[0, 1947, 2792, 3837, 0, 0, 1975]
[0, 1799, 2990, 3937, 0, 0, 1942]
[0, 1506, 3565, 4245, 0, 0, 1757]
[0, 0, 3873, 4308, 0, 0, 1404]
[0, 1963, 3942, 4341, 0, 0, 0]
[0, 1963, 3942, 4341, 0, 0, 0]
[0, 0, 4254, 4346, 0, 0, 0]
[0, 0, 2127, 2873, 0, 0, 1365]
[0, 0, 0, 3201, 0, 0, 2003]
[0, 0, 0, 3358, 0, 0, 1819]
[0, 0, 0, 3938, 0, 0, 1302]
[0, 0, 0, 4317, 1861, 0, 0]
[0, 0, 0, 4317, 1861, 0, 0]
[0, 0, 1879, 4414, 1709, 0, 0]
[0, 0, 2108, 0, 0, 0, 2090]
[0, 0, 2617, 0, 0, 0, 2164]
[0, 2688, 2479, 0, 0, 0, 2172]
[0, 2593, 2658, 0, 0, 0, 2136]
[0, 2341, 3009, 0, 0, 0, 1904]
[0, 2943, 3704, 0, 0, 0, 1671]
[0, 3492, 4136, 0, 0, 0, 0]
[0, 3492, 4136, 0, 0, 0, 0]
[0, 3808, 4187, 0, 0, 0, 0]
[0, 4235, 4159, 0, 0, 0, 0]
[0, 4299, 380

[0, 0, 2325, 0, 0, 0, 0]
[0, 0, 1749, 0, 0, 0, 0]
[0, 0, 1874, 0, 0, 0, 0]
[0, 0, 2103, 0, 0, 0, 0]
[0, 0, 2340, 0, 0, 0, 0]
[0, 0, 2773, 0, 0, 0, 0]
[0, 0, 2957, 0, 0, 0, 0]
[0, 0, 3329, 0, 0, 0, 0]
[0, 0, 3800, 0, 0, 0, 0]
[0, 0, 4147, 1831, 0, 0, 0]
[0, 0, 4147, 1831, 0, 0, 0]
[0, 0, 4193, 2145, 0, 0, 0]
[0, 0, 4212, 2762, 0, 0, 0]
[0, 0, 4187, 3683, 0, 0, 0]
[0, 0, 4017, 4348, 0, 0, 0]
[0, 0, 3295, 4433, 0, 0, 0]
[0, 0, 2286, 4447, 0, 0, 0]
[0, 0, 0, 4444, 3290, 0, 0]
[0, 0, 0, 4444, 3290, 0, 0]
[0, 0, 0, 4408, 4113, 0, 0]
[0, 0, 0, 4098, 4209, 0, 0]
[0, 0, 0, 3102, 4230, 0, 0]
[0, 0, 0, 1757, 4237, 0, 0]
[0, 0, 0, 0, 4224, 0, 0]
[0, 0, 0, 0, 4185, 0, 0]
[0, 0, 0, 0, 3876, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 2942, 0, 0]
[0, 0, 0, 1768, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[4233, 4219, 0, 0, 0, 0, 0]
[4233, 421

KeyboardInterrupt: 

In [90]:
sensor_measurment = robot["prox.horizontal"]
obstacle_direction = np.nonzero(sensor_measurment)[0]

print(sensor_measurment)
print(obstacle_direction)
#print(np.argmax(sensor_measurment))
print(np.shape(obstacle_direction)[0])
print(np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))
print(np.all(np.not_equal(obstacle_direction, SENSOR_5)))
print(np.all(np.not_equal(obstacle_direction, SENSOR_6)))
print(np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)))

#robot.set_var("motor.left.target", 50)
#robot.set_var("motor.right.target", 2**16-50)
#time.sleep(1.1)
robot.set_var("motor.left.target", 0)
robot.set_var("motor.right.target", 0)

[0, 0, 0, 0, 0, 0, 0]
[]
0
False
True
True
True


Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\Users\roxan\Documents\Applications\Anaconda\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\roxan\Documents\Applications\Anaconda\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\roxan\Documents\EPFL_COURS\Basics of mobile robotics\Solutions_4\Solutions\src\Thymio.py", line 340, in do_refresh
    self.get_variables()
  File "C:\Users\roxan\Documents\EPFL_COURS\Basics of mobile robotics\Solutions_4\Solutions\src\Thymio.py", line 499, in get_variables
    self.send(msg)
  File "C:\Users\roxan\Documents\EPFL_COURS\Basics of mobile robotics\Solutions_4\Solutions\src\Thymio.py", line 456, in send
    self.io.write(msg.serialize())
  File "C:\Users\roxan\Documents\Applications\Anaconda\lib\site-packages\serial\serialwin32.py", line 315, in write
    raise SerialException("WriteFile failed ({!r})".format(ctypes.WinError()))
serial.seri

### Bugs to fix

1) if the robot meet a v shape obstacle  
2) if the robot has an obstacle in front of sensor n°2 it can turns and then sense the obstacle on the 3th, it turns backwards --> wait a bit more in the case of detection on the sensor 2 ?